In [350]:
import pandas as pd
import numpy as np 
import os

<h2>Load Data</h2>

In [351]:
credits_df = pd.read_csv(os.path.join(os.getcwd(), 'datasets/movies/credits.csv'))
keywords_df = pd.read_csv(os.path.join(os.getcwd(), 'datasets/movies/keywords.csv'))
links_df = pd.read_csv(os.path.join(os.getcwd(), 'datasets/movies/links.csv'), dtype={"imdbId":"string"})
movies_df = pd.read_csv(os.path.join(os.getcwd(), 'datasets/movies/movies_metadata.csv'))
ratings_df = pd.read_csv(os.path.join(os.getcwd(), 'datasets/movies/ratings.csv'))

C:\Users\leish\AppData\Local\Temp\ipykernel_27112\4291322724.py:4: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv(os.path.join(os.getcwd(), 'datasets/movies/movies_metadata.csv'))


In [352]:
links_df.drop_duplicates(), links_df.shape

(       movieId   imdbId    tmdbId
 0            1  0114709     862.0
 1            2  0113497    8844.0
 2            3  0113228   15602.0
 3            4  0114885   31357.0
 4            5  0113041   11862.0
 ...        ...      ...       ...
 45838   176269  6209470  439050.0
 45839   176271  2028550  111109.0
 45840   176273  0303758   67758.0
 45841   176275  0008536  227506.0
 45842   176279  6980792  461257.0
 
 [45843 rows x 3 columns],
 (45843, 3))

<h2>Fix Type Issues</h2>

In [353]:
movies_df['id'] = pd.to_numeric(movies_df['id'], errors='coerce')
movies_df = movies_df[movies_df['id'].notna()].astype({'id': int})

In [354]:
movies_df.shape

(45463, 24)

<h2>Add Wikipedia Links to links.csv</h2>

In [355]:
from SPARQLWrapper import SPARQLWrapper, JSON

In [356]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

sparql.setQuery("""SELECT ?item ?IMDb_ID ?sitelink WHERE {

              {
                {
                ?item wdt:P31 /wdt:P279* wd:Q11424 .
              ?item wdt:P345 ?IMDb_ID .
              ?sitelink schema:about ?item ; schema:isPartOf <https://en.wikipedia.org/> .
              }
              }
            UNION {
              ?item wdt:P31 /wdt:P279* wd:Q1259759 .
              ?item wdt:P345 ?IMDb_ID .
              ?sitelink schema:about ?item ; schema:isPartOf <https://en.wikipedia.org/> .
            }
                  

            }""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
results_df = pd.json_normalize(results['results']['bindings'])
results_df = results_df[['IMDb_ID.value','sitelink.value']]
results_df.rename(columns={'IMDb_ID.value':'imdbId', 'sitelink.value':"wikipediaId"}, inplace=True)
results_df.drop_duplicates(inplace=True)


In [357]:
results_df.loc[results_df.imdbId == "tt5013056"]

,imdbId,wikipediaId
109274,tt5013056,https://en.wikipedia.org/wiki/Dunkirk_(2017_film)


In [358]:
links_df['imdbId'] = np.where("tt" in links_df['imdbId'].astype(str), links_df['imdbId'].astype(str), "tt"+links_df['imdbId'].astype(str))
#links_df['movieId'] = movies_df.loc[movies_df.imdb_id == links_df.imdbId, movies_df.id]
links_df = links_df.merge(results_df, on="imdbId")

links_df.rename(columns={'movieId':'id', 'imdbId':'imdb_id'},inplace=True)
#links_df.drop_duplicates(inplace=True)

<h2>Calculate Average Rating for Movie</h2>

In [359]:
grouped_ratings_df = ratings_df.groupby('movieId').mean()
grouped_ratings_df.reset_index(inplace=True)
grouped_ratings_df.rename(columns={"movieId":"id"}, inplace=True)

In [360]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'keywords', 'cast', 'crew', 'rating',
       'wikipediaId'],
      dtype='object')

<h2>Merge Datasets to form a Master Dataset</h2>

In [361]:
df = movies_df.merge(keywords_df, how='left', on='id')
print(df.shape)
df = df.merge(credits_df[['cast','crew','id']], how='left', on='id')
print(df.shape)
df = df.merge(grouped_ratings_df[['id','rating']], how='left', on='id')
print(df.shape)
df = df.merge(links_df[['imdb_id','wikipediaId']], how='left', on='imdb_id')
print(df.shape)

(46483, 25)
(46629, 27)
(46629, 28)
(46634, 29)


In [362]:
df[df['wikipediaId'].isna()].loc[df.vote_count > 500]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,tagline,title,video,vote_average,vote_count,keywords,cast,crew,rating,wikipediaId
22872,False,"{'id': 258664, 'name': 'Nymphomaniac Collectio...",0,"[{'id': 18, 'name': 'Drama'}, {'id': 9648, 'na...",http://www.magpictures.com/nymphomaniac/,249397,tt2382009,da,Nymphomaniac: Vol. II,The continuation of Joe's sexually dictated li...,...,Forget About Love,Nymphomaniac: Vol. II,False,6.6,820.0,"[{'id': 572, 'name': 'sex'}, {'id': 910, 'name...","[{'cast_id': 1, 'character': 'Joe', 'credit_id...","[{'credit_id': '52fe4f43c3a36847f82c9289', 'de...",NaN,NaN


<h2>Expands cells of master dataframe into smaller dataframes</h2>

In [363]:
# Creates seperate dataframe that contains all information for a specific column of that dataframe
# The column converts the JSON or array of JSONs of each row into a row to be inserted into the DataFrame
def create_info_dataframe(df,column_header):
    if "[" in df[df[column_header].notna()][column_header].iloc[0]:  
        return pd.DataFrame([x for sublist in df[column_header].map(lambda x: eval(x) if isinstance(x,str) else "").to_numpy().flatten() for x in sublist]).drop_duplicates()
    else:
        return pd.DataFrame([x for x in df[column_header].map(lambda x: eval(x) if isinstance(x, str) and isinstance(eval(x), dict) else eval(
            "{'id': 0,'name':'None','poster_path':'None', 'backdrop_path':'None'}"))])

<h2>Simplifies complicated cells into an array of IDs</h2>

In [364]:
# Extracts the id of each JSON in each cell and creates an array of these ids to be inserted into the original DataFrame df. 
# The Id of the JSON can reference the additional DataFramescd 
def reduce_json_to_id(df, column_header):
    if "[" in df[df[column_header].notna()][column_header].iloc[0]: 
        return df[column_header].map(lambda x: [y['id'] for y in eval(x)] if isinstance(x, str) else None)
    else:
        return df[column_header].map(lambda x: eval(x)['id'] if isinstance(x, str) and isinstance(eval(x), dict) else None)

In [365]:
genres_df = create_info_dataframe(df, 'genres')
keywords2_df = create_info_dataframe(df, 'keywords')
cast_df = create_info_dataframe(df, 'cast')
crew_df = create_info_dataframe(df, 'crew')
collection_df = create_info_dataframe(df, 'belongs_to_collection')
production_company_df = create_info_dataframe(df, 'production_companies')

In [366]:
df['genres'] = reduce_json_to_id(df, 'genres')
df['keywords'] = reduce_json_to_id(df, 'keywords')
df['cast'] = reduce_json_to_id(df, 'cast')
df['crew'] = reduce_json_to_id(df, 'crew')
df['belongs_to_collection'] = reduce_json_to_id(df, 'belongs_to_collection')
df['production_companies'] = reduce_json_to_id(df, 'production_companies')

In [367]:
df = df[~df['wikipediaId'].isna()]

In [368]:
df.to_csv(os.path.join(os.getcwd(), 'datasets/movies/master.csv'))
genres_df.to_csv(os.path.join(os.getcwd(), 'datasets/movies/genres.csv'))
cast_df.to_csv(os.path.join(os.getcwd(), 'datasets/movies/cast.csv'))
crew_df.to_csv(os.path.join(os.getcwd(), 'datasets/movies/crew.csv'))
collection_df.to_csv(os.path.join(os.getcwd(), 'datasets/movies/collections.csv'))
production_company_df.to_csv(os.path.join(os.getcwd(), 'datasets/movies/production_companies.csv'))
keywords2_df.to_csv(os.path.join(os.getcwd(), 'datasets/movies/keywords2.csv'))

In [369]:
df.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'keywords', 'cast', 'crew', 'rating',
       'wikipediaId'],
      dtype='object')